In [21]:
import json
import pandas as pd
from PortfolioTracker.GlobalPath import GlobalPath

# Instantiate GlobalPath
global_path = GlobalPath("PortfolioTracker")
# GLOBAL PATH
profitloss_gold_file_path = global_path.joinpath(
    "DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)
## PRESENTATION LAYER
df_profitloss = pd.read_csv(profitloss_gold_file_path)
# Convert datetime to date string
df_profitloss["open_datetime"] = pd.to_datetime(df_profitloss["open_datetime"]).dt.date
df_profitloss["close_datetime"] = pd.to_datetime(df_profitloss["close_datetime"]).dt.date
# Grouping by 'name' and 'type'
result = []
grouped = df_profitloss.groupby(["segment", "exchange", "symbol"])
    
for (segment, exchange, symbol), group in grouped:
    group_dict = {
        "segment": segment,
        "exchange": exchange,
        "symbol": symbol,
        "days": (group["close_datetime"].max()-group["open_datetime"].min()).days,
        "quantity": group["quantity"].sum(),
        "avg_price": group["open_amount"].sum() / group["quantity"].sum(),
        "sell_price": group["close_amount"].sum() / group["quantity"].sum(),
        "pnl": group["pnl_amount"].sum(),
        "history": group[
            [
                "stock_name",
                "position",
                "quantity",
                "open_datetime",
                "open_side",
                "open_price",
                "open_amount",
                "close_datetime",
                "close_side",
                "close_price",
                "close_amount",
                "pnl_amount",
                "pnl_percentage",
            ]
        ].to_dict(orient="records"),
    }
    result.append(group_dict)

output = {
    "invested": df_profitloss["open_amount"].sum(),
    "sold": df_profitloss["close_amount"].sum(),
    "pnl": df_profitloss["pnl_amount"].sum(),
    "data": result,
}

# Writing the result to a JSON file
output_path = global_path.joinpath("DATA/GOLD/ProfitLoss/ProfitLoss_data.json")
with open(output_path, "w") as json_file:
    json.dump(output, json_file, indent=4,default=str)

print(f"Data written to {output_path.resolve()}")

Data written to C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\GOLD\ProfitLoss\ProfitLoss_data.json
